In [ ]:
# Function (tools) calling

In [ ]:
from dotenv import load_dotenv
from rich import print

%load_ext autoreload
%autoreload 2

load_dotenv(verbose=True)

In [ ]:
from langchain_core.tools import tool

from src.ai_core.llm import get_llm
from src.ai_core.prompts import def_prompt

### Step 1 : Ask LLM to return an instantiated function  

idea : call https://v2.jokeapi.dev/
https://github.com/benjhar/JokeAPI-Python#readme


In [ ]:
# llm = get_llm("gpt_4o_azure")
llm = get_llm(None)
messages = []


@tool
def add(a: int, b: int) -> int:
    """Addition 2 integer numbers a and b.

    Args:
        a: first int
        b: second int
    """
    print("'add' tool called")
    return a + b


llm_with_tools = llm.bind_tools([add], tool_choice="any")  # tool_choice="any"
prompt = def_prompt(
    "Use the provided functions  to compute what is 45 + 12; execute the function. Return 'I don't know' if there are not relevant function  "
)
chain = prompt | llm_with_tools
ai_msg = chain.invoke({})

print(ai_msg)

In [ ]:
print(llm_with_tools)

In [ ]:
# Step 2 : Execute the function, and send result to LLM

###  Use a tool calling agent

In [ ]:
import math

from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
from langchain_core.prompts import ChatPromptTemplate


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def exponentiate(x: float, y: float) -> float:
    """Calculate the power of a number. Return x**y (w to the power of y)."""
    print("exponentiate")
    return math.pow(x, y)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful Math Assistant. Please use the provided tool"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

tools = [add, exponentiate]
agent = create_tool_calling_agent(llm, tools, prompt)  # type: ignore
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)  # type: ignore

r = agent_executor.invoke({"input": "what is 12  + 100^3"})
print(r)